<a href="https://colab.research.google.com/github/azank1/llmshi/blob/main/Copy_of_Lab_3_rag_lab_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3:Introduction to RAG

## Task 1: Requirements and Setup

Azan Hyder #365860


- Install the required libraries and ensure GPU access in Google Colab.

In [ ]:
# Perform Google Colab installs (if running in Google Colab)
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    # REMOVED flash-attn to prevent build errors

[INFO] Running in Google Colab, installing requirements.


## Task 2: Import PDF Document

- Import the PDF file and extract its pages using PyMuPDF.

In [ ]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "human-nutrition-text.pdf"

# Download PDF if it doesn't already exist
if not os.path.exists(pdf_path):
  print("File doesn't exist, downloading...")

  # The URL of the PDF you want to download
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  # The local filename to save the downloaded file
  filename = pdf_path

  # Send a GET request to the URL
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Open a file in binary write mode and save the content to it
      with open(filename, "wb") as file:
          file.write(response.content)
      print(f"The file has been downloaded and saved as {filename}")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print(f"File {pdf_path} exists.")

File doesn't exist, downloading...
The file has been downloaded and saved as human-nutrition-text.pdf


In [ ]:
# Requires !pip install PyMuPDF, see: https://github.com/pymupdf/pymupdf
import fitz # (pymupdf, found this is better than pypdf for our use case, note: licence is AGPL-3.0, keep that in mind if you want to use any code commercially)
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages

    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

IndentationError: expected an indented block after 'for' statement on line 28 (ipython-input-81512617.py, line 30)

## Task 3: Sample Pages

- Select and display a random sample of pages from the document.

In [ ]:
import random

random.sample(pages_and_texts, k=3)

## Task 4: Text Statistics

- Compute and display descriptive statistics (e.g., character counts, word counts) of the text pages.

In [ ]:
import pandas as pd

df =
df.head()

In [ ]:
# Get stats
df.describe().round(2)

## Task 5: Sentence Tokenization

- Split the text pages into sentences using spaCy or an equivalent library.

In [ ]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/


# Create a document instance as an example


# Access the sentences of the document
list(doc.sents)

In [ ]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings


    # Count the sentences


In [ ]:
# Inspect an example
random.sample(pages_and_texts, k=1)

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

## Task 6: Chunk Sentences

- Group sentences into chunks of a specified size.

In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

In [ ]:
# Sample an example from the group (note: many samples have only 1 chunk as they have <=10 sentences total)
random.sample(pages_and_texts, k=1)

In [ ]:
# Create a DataFrame to get stats
df =
df.describe().round(2)

## Task 7: Split Chunks

- Split each sentence chunk into individual records for embedding preparation.

In [ ]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)


        # Get stats about the chunk


        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

In [ ]:
# View a random sample
random.sample(pages_and_chunks, k=1)

In [ ]:
# Get stats about our chunks
df =
df.describe().round(2)

## Task 8: Filter Small Chunks

- Filter out text chunks below a minimum token length threshold.

In [ ]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

## Task 9: Embed Text Chunks

- Embed each text chunk into numeric vectors using a sentence-transformer model.

In [ ]:
# Requires !pip install sentence-transformers
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cuda") # choose the device to load the model to (note: GPU will often be *much* faster than CPU)

# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()


# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

In [ ]:
single_sentence = "Yo! How cool are embeddings?"
single_embedding = embedding_model.encode(single_sentence)
print(f"Sentence: {single_sentence}")
print(f"Embedding:\n{single_embedding}")
print(f"Embedding size: {single_embedding.shape}")

## Task 10: Batch Embedding on GPU

- Execute batched embeddings on GPU to improve performance and measure speed.

In [ ]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [ ]:
%%time

# Embed all texts in batches
text_chunk_embeddings =

text_chunk_embeddings

## Task 11: Save and Load Embeddings

- Save the embeddings to a CSV file and reload it to verify integrity.

In [ ]:
# Save embeddings to file


In [ ]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

## Task 12: Build Retrieval System

- **Load embeddings**  
  Read `text_chunks_and_embeddings_df.csv` into a pandas DataFrame.

- **Create a vector index**  
  Initialize a FAISS (or similar) index with the chunk embeddings.

- **Index all chunks**  
  Add every embedding vector to your index to prepare for similarity search.

- **Implement a retrieval function** that:
  1. Takes a user query (string) as input.  
  2. Embeds it using the same sentence‑transformer model.  
  3. Runs a top‑k similarity search (e.g. k = 5) against your FAISS index.  
  4. Returns the corresponding text chunks.

- **Test your pipeline**  
  Try queries like “What is the recommended daily protein intake?” and inspect the retrieved passages.



In [ ]:
#Load the CSV of chunks + embeddings


# 1. Load the same model you used for embedding


# 2. Embed all sentence chunks in batches


# 3. Attach embeddings back to DataFrame


In [ ]:
!pip install faiss-cpu -q
import faiss

# 1. Get embedding dimension
dim = df["embedding"].iloc[0].shape[0]

# 2. Create a flat L2 index
index = faiss.IndexFlatL2(dim)

# 3. Add all embeddings to the index
index.add(np.vstack(df["embedding"].to_list()))
print("FAISS index: n_vectors =", index.ntotal)


In [ ]:

def retrieve(query: str, k: int = 5):
    # 1. Embed the query
    q_emb = model.encode(query).astype("float32").reshape(1, -1)
    # 2. Search for top-k
    dists, idxs = index.search(q_emb, k)
    # 3. Collect results
    return [
        {
            "page": int(df.iloc[i]["page_number"]),
            "chunk": df.iloc[i]["sentence_chunk"],
            "distance": float(d)
        }
        for d, i in zip(dists[0], idxs[0])
    ]

# Example usage
for hit in retrieve("What is the recommended daily intake of protein?"):
    print(f"[Pg {hit['page']}] (dist={hit['distance']:.3f}) {hit['chunk']}")


## Task 13: Questions

- Answer the following questions regarding embedding model selection and chunking:

1. Which embedding model should you use and why?

2. How might chunk size affect embedding performance?

3. Why is chunking necessary in a RAG pipeline, and what would happen if we embedded entire documents instead?

4. Why must the same embedding model be used for both document chunks and user queries?

5. Why does this lab use vector similarity search (FAISS) instead of keyword matching or rule-based search?

6. Can a RAG system still return incorrect or misleading answers even if retrieval works correctly? Explain why.

# Task
Install the required libraries (`torch`, `PyMuPDF`, `tqdm`, `sentence-transformers`, `accelerate`, `bitsandbytes`, and `flash-attn`) and verify that all installations are successful and the environment is ready for further tasks, including confirming GPU access.

## Install Requirements

### Subtask:
Execute the code cell to install the required libraries like `torch`, `PyMuPDF`, `tqdm`, `sentence-transformers`, `accelerate`, `bitsandbytes`, and `flash-attn`. This step will also check for GPU access.


**Reasoning**:
The subtask requires executing the provided code cell to install necessary libraries and check for GPU access. The code cell `277c519c` performs exactly this action.



In [ ]:
# Perform G
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference

**Reasoning**:
The subtask explicitly states to execute the provided code cell to install necessary libraries and check for GPU access. The code cell `277c519c` is designed for this purpose.



In [ ]:
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference

[INFO] Running in Google Colab, installing requirements.
  Using cached pymupdf-1.26.7-cp310-abi3-manylinux_2_28_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.7-cp310-abi3-manylinux_2_28_x86_64.whl (24.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 95.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)
